In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import pymysql

In [43]:
# Functions

# reverse the data of argument
def reverseData(data):
    data = data[::-1]
    return data

def getRound(data, decimals = 3):
    return np.around(data, decimals)

## given two curve a, b, then return the inversection point of two curves
def FindTheBuyPoint(a, b, mode):
    if(mode == "rsi_1"):
        buyPoint = np.argwhere(np.diff(np.sign(a - 20)) > 0).reshape(-1) + 0
    else:
        buyPoint = np.argwhere(np.diff(np.sign(a - b)) > 0).reshape(-1) + 0
    return buyPoint

def FindTheSellPoint(a, b, mode):
    if(mode == "rsi_1"):
        sellPoint = np.argwhere(np.diff(np.sign(a - 80)) < 0).reshape(-1) + 0
    else:
        sellPoint = np.argwhere(np.diff(np.sign(a - b)) < 0).reshape(-1) + 0
    return sellPoint

def GetDataFromDB(ip, userName, psw, tableName, sql):
    try:
        db = pymysql.connect(ip, userName, psw, tableName)
        #create a cursor
        cursor = db.cursor()
        results = []
        try:
            cursor.execute(sql)
            results = cursor.fetchall()
        except:
            print ("Error: unable to fetch data from DB")
        db.close()
        return results
    except:
        print ("Error: unable to connect to DB")
    
def DeterminedStatus(indicatorResultFast, indicatorResultSlow, result):
    longShortIndex = [i for i in range(len(result)) if result[i] == 1 or result[i] == -1]
    for i in range(len(longShortIndex) - 1):
        begin = longShortIndex[i]
        end = longShortIndex[i + 1]
        # find the series between begin and end
        selectedIndicatorResultFast = indicatorResultFast[begin + 1:end]
        if len(selectedIndicatorResultFast) == 0:
            continue
        selectedIndicatorResultSlow = indicatorResultSlow[begin + 1:end]
        if len(selectedIndicatorResultSlow) == 0:
            continue
        # find the distance of every elements
        distanceSeries = [abs(x1 - x2) for (x1, x2) in zip(selectedIndicatorResultFast, selectedIndicatorResultSlow)]
        
        # normalize all distance to [0, 1]
        maxDistance = max(distanceSeries)
        distanceSeries = [i * 2 / maxDistance for i in distanceSeries]
        
        # find the index of largest distance
        largestIndex = begin + 1 + distanceSeries.index(2)
        result[largestIndex] = 0
        firstNumber, lastNumber = result[begin], result[end]
        for j in range(begin + 1, end):
            if j != largestIndex:
                if firstNumber == 1:
                    result[j] = firstNumber - distanceSeries[j - (begin + 1)]
                else:
                    result[j] = firstNumber + distanceSeries[j - (begin + 1)]
    # TO deal w/ the records which are not between long point and short point
    begin = longShortIndex[-1]
    end = len(indicatorResultFast)
    targetListF = indicatorResultFast[begin + 1:end]
    targetListS = indicatorResultSlow[begin + 1:end]
    # find the distance of every elements
    distanceSeries = [abs(x1 - x2) for (x1, x2) in zip(targetListF, targetListS)]
    print ('distanceSeries')
    print (distanceSeries)
    # normalize all distance to [0, 1]
    maxDistance = max(distanceSeries)
    distanceSeries = [i * 2 / maxDistance for i in distanceSeries]
    print ('modified distanceSeries')
    print (distanceSeries)
    # find the index of largest distance
    largestIndex = begin + 1 + distanceSeries.index(2)
    result[largestIndex] = 0
    firstNumber, lastNumber = result[begin], result[end - 1]
    for i in range(begin + 1, end):
        if i != largestIndex:
            if firstNumber == 1:
                result[i] = firstNumber - distanceSeries[i - (begin + 1)]
            else:
                result[i] = firstNumber + distanceSeries[i - (begin + 1)]
    return result

def getFeaturesCrossIndicators(historicalPrices):
    indicatorNameList = [
        'sma5',
        'sma10',
        'sma20',
        'sma60',
        'sma120',
        'sma240'
    ]
    indicators = {}
    indicators['sma5'] = SMA(historicalPrices['Close'], 5)
    indicators['sma10'] = SMA(historicalPrices['Close'], 10)
    indicators['sma20'] = SMA(historicalPrices['Close'], 20)
    indicators['sma60'] = SMA(historicalPrices['Close'], 60)
    indicators['sma120'] = SMA(historicalPrices['Close'], 120)
    indicators['sma240'] = SMA(historicalPrices['Close'], 240)
    indicators['bias5'] = Bias(historicalPrices['Close'], 5)
    indicators['bias10'] = Bias(historicalPrices['Close'], 10)
    indicators['bias20'] = Bias(historicalPrices['Close'], 20)
    indicators['bias60'] = Bias(historicalPrices['Close'], 60)
    indicators['bias120'] = Bias(historicalPrices['Close'], 120)
    indicators['bias240'] = Bias(historicalPrices['Close'], 240)
    indicators['rsi5'] = RSI(historicalPrices['Close'], 5)
    indicators['rsi14'] = RSI(historicalPrices['Close'], 14)
    kd = KD(historicalPrices['Close'], historicalPrices['High'], historicalPrices['Low'])
    indicators['k'] = kd['K9']
    indicators['d'] = kd['D9']
    macd = MACD(historicalPrices['Close'], historicalPrices['High'], historicalPrices['Low'], [12, 26, 9])
    indicators['dif'] = macd['dif']
    indicators['dem'] = macd['dem']
    
    crossOverTwoIndicators = {}
    for i in range(len(indicatorNameList) - 1):
        for j in range(i + 1, len(indicatorNameList)):
            crossOverTwoIndicators[indicatorNameList[i] + '-' + indicatorNameList[j]] = indicators[indicatorNameList[i]] - indicators[indicatorNameList[j]]
    crossOverTwoIndicators['sma5-close'] = indicators['sma5'] - historicalPrices['Close']
    crossOverTwoIndicators['sma10-close'] = indicators['sma10'] - historicalPrices['Close']
    crossOverTwoIndicators['sma20-close'] = indicators['sma20'] - historicalPrices['Close']
    crossOverTwoIndicators['sma60-close'] = indicators['sma60'] - historicalPrices['Close']
    crossOverTwoIndicators['sma120-close'] = indicators['sma120'] - historicalPrices['Close']
    crossOverTwoIndicators['sma240-close'] = indicators['sma240'] - historicalPrices['Close']
    
    indicatorNameList = [
        'bias5',
        'bias10',
        'bias20',
        'bias60',
        'bias120',
        'bias240'
    ]
    for indicatorName in indicatorNameList:
        for index in range(1,11):
            crossOverTwoIndicators['{indicatorName}-({num}%)'.format(indicatorName = indicatorName, num = index)] = indicators[indicatorName] - index
            crossOverTwoIndicators['{indicatorName}-(-{num}%)'.format(indicatorName = indicatorName, num = index)] = indicators[indicatorName] + index
    
    crossOverTwoIndicators['rsi5-30'] = indicators['rsi5'] - pd.Series([30 for i in range(len(indicators['rsi5']))])
    crossOverTwoIndicators['rsi5-70'] = indicators['rsi5'] - pd.Series([70 for i in range(len(indicators['rsi5']))])
    crossOverTwoIndicators['rsi5-20'] = indicators['rsi5'] - pd.Series([20 for i in range(len(indicators['rsi5']))])
    crossOverTwoIndicators['rsi5-80'] = indicators['rsi5'] - pd.Series([80 for i in range(len(indicators['rsi5']))])
    crossOverTwoIndicators['rsi14-30'] = indicators['rsi14'] - pd.Series([30 for i in range(len(indicators['rsi14']))])
    crossOverTwoIndicators['rsi14-70'] = indicators['rsi14'] - pd.Series([70 for i in range(len(indicators['rsi14']))])
    crossOverTwoIndicators['rsi14-20'] = indicators['rsi14'] - pd.Series([20 for i in range(len(indicators['rsi14']))])
    crossOverTwoIndicators['rsi14-80'] = indicators['rsi14'] - pd.Series([80 for i in range(len(indicators['rsi14']))])
    crossOverTwoIndicators['rsi5-rsi14'] = indicators['rsi5'] - indicators['rsi14']
    
    crossOverTwoIndicators['k-d'] = indicators['k'] - indicators['d']
    crossOverTwoIndicators['k-30'] = indicators['k'] - pd.Series([30 for i in range(len(indicators['rsi5']))])
    crossOverTwoIndicators['k-70'] = indicators['k'] - pd.Series([70 for i in range(len(indicators['rsi5']))])
    crossOverTwoIndicators['k-20'] = indicators['k'] - pd.Series([20 for i in range(len(indicators['rsi5']))])
    crossOverTwoIndicators['k-80'] = indicators['k'] - pd.Series([80 for i in range(len(indicators['rsi5']))])
    
    crossOverTwoIndicators['dif-dem'] = indicators['dif'] - indicators['dem']
    return pd.DataFrame(crossOverTwoIndicators)

In [33]:
# indicators
def SMA(closePrices, period):
    if (type(closePrices) == pd.core.series.Series):
        return closePrices.rolling(window = period).mean()
    else:
        print ("Error: Wrong input, SMA(pandas.core.series.Series, integer)")
        
def EMA(closePrices, period):
    if (type(closePrices) == pd.core.series.Series):
        return closePrices.ewm(span = period, adjust = False).mean()
    else:
        print ("Error: Wrong input, EMA(pandas.core.series.Series, integer)")
        
def RSI(closePrices, period):
    if (type(closePrices) == pd.core.series.Series):
        delta = closePrices.diff()
        up, down = delta.copy(), delta.copy()

        up[up < 0] = 0
        down[down > 0] = 0
        down = abs(down)
        
        Up = SMA(up, period)
        Up = Up.fillna(0)
        for i in range(period,len(Up)):
            Up[i] = getRound((Up[i - 1] * (period - 1) + up[i]) / period)

        Down = SMA(down, period)
        Down = Down.fillna(0)
        for i in range(period,len(Down)):
            Down[i] = getRound((Down[i - 1] * (period - 1) + down[i]) / period)


        rsi = 100 * (Up / (Up + Down))
        rsi = rsi.fillna(0)
        return getRound(rsi, 2)
    else:
        print ("Error: Wrong input, RSI(pandas.core.series.Series, integer)")
        
def MACD(closePrices, highPrices, lowPrices, period = []):
    if (type(closePrices) == pd.core.series.Series and type(highPrices) == pd.core.series.Series and type(lowPrices) == pd.core.series.Series and len(period) == 3):   
        di = (highPrices + lowPrices + 2.0 * closePrices) / 4.0
        ema12 = SMA(di, period[0])
        ema12 = ema12.fillna(0)
        for i in range(period[0] + 1, len(ema12)):
            ema12[i] = (ema12[i - 1] * (period[0] - 1) + di[i] * 2.0) / (period[0] + 1)
    
        ema26 = SMA(di, period[1])
        ema26 = ema26.fillna(0)
        for i in range(period[1] + 1, len(ema26)):
            ema26[i] = (ema26[i - 1] * (period[1] - 1) + di[i] * 2.0) / (period[1] + 1)
    
        dif = ema12 - ema26

        dem = SMA(dif, period[2])
        dem = dem.fillna(0)
        for i in range(period[2] + 1, len(dem)):
            dem[i] = (dem[i - 1] * (period[2] - 1) + dif[i] * 2.0) / (period[2] + 1)
        return {'dif':dif, 'dem':dem}
    else:
        if(type(data) == pd.core.frame.DataFrame):
            print ("Error: Wrong input, MACD(pandas.core.frame.DataFrame, list of integer)")
        elif(len(period) == 3):
            print ("Error: number of content in list do not equal to 3")
            
def KD(closePrices, highPrices, lowPrices, result = {}):
    close = closePrices.copy()
    for i in range(0,9):
        close[i] = 0
    data = pd.DataFrame()
    data['RSV'] = (( closePrices - lowPrices.rolling(window = 9).min()) / (highPrices.rolling(window = 9).max() - lowPrices.rolling(window = 9).min()))
    data['RSV'] = data['RSV'].fillna(0)
    if(not result):
        result = {
            'K9':[0],
            'D9' :[0]
        }
    #calculate everyday's KD
    for i in range(1, len(data.index)):
        K9_value = (1.0/3.0) * data['RSV'][i] + (2.0 / 3.0) * result['K9'][i - 1]
        result['K9'].append(getRound(K9_value, 5))
        D9_value = (2.0/3.0) * result['D9'][i - 1] + (1.0 / 3.0) * result['K9'][i]
        result['D9'].append(getRound(D9_value, 5))
        
    return pd.DataFrame(result)

def Bias(closePrices, period):
    if not isinstance(closePrices, pd.core.series.Series) or not isinstance(period, int):
        print("Undefined type")
        return None
    sma = SMA(closePrices=closePrices, period=period)
    return (closePrices / sma - 1) * 100

In [56]:
# get data
etfPriceData = pd.read_csv(r"C:\Users\user\Desktop\TBrain_Round2_DataSet_20180615\taetfp.csv", encoding="big5")

In [57]:
# kd
KD_val = pd.DataFrame(KD(etfPriceData))
# kd's long points & short points
KD_long = FindTheBuyPoint(KD_val['K9'], KD_val['D9'], "kd")
KD_short = FindTheSellPoint(KD_val['K9'], KD_val['D9'], "kd")

# create a long/short points list
KD_result = [0 for i in range(len(KD_val))]

for i in KD_long:
    KD_result[i] = 1
for i in KD_short:
    KD_result[i] = -1
    
KD_result = DeterminedStatus(list(KD_val['K9']), list(KD_val['D9']), KD_result)
KD_result = pd.Series(KD_result).fillna(0).tolist()

In [58]:
# rsi
rsi5_result = pd.DataFrame(RSI(etfPriceData['收盤價(元)'], 5))
rsi5_result = rsi5_result.rename(columns={'收盤價(元)': 'rsi'})

rsi14_result = pd.DataFrame(RSI(etfPriceData['收盤價(元)'], 14))
rsi14_result = rsi14_result.rename(columns={'收盤價(元)': 'rsi'})

# rsi's long points & short points
rsi_long_with1curve = FindTheBuyPoint(rsi14_result['rsi'], [], "rsi_1")
rsi_short_with1curve = FindTheSellPoint(rsi14_result['rsi'], [], "rsi_1")

rsi_long_with2curve = FindTheBuyPoint(rsi5_result['rsi'], rsi14_result['rsi'], "rsi_2")
rsi_short_with2curve = FindTheSellPoint(rsi5_result['rsi'], rsi14_result['rsi'], "rsi_2")

# create a long/short points list
rsi1curve_result = [0 for i in range(len(rsi14_result))]
rsi2curve_result = [0 for i in range(len(rsi14_result))]

for i in rsi_long_with1curve:
    rsi1curve_result[i] = 1
for i in rsi_short_with1curve:
    rsi1curve_result[i] = -1
for i in range(len(rsi1curve_result)):
    if rsi1curve_result[i] == 0:
        if rsi14_result['rsi'][i] >= 20 and rsi14_result['rsi'][i] <= 80:
            rsi1curve_result[i] = -1.0 + 2.0 * (rsi14_result['rsi'][i] - 20.0) / (80.0 - 20.0)
        elif rsi14_result['rsi'][i] < 20:
            rsi1curve_result[i] = 1 + abs((rsi14_result['rsi'][i] - 20.0) / 20)
        elif rsi14_result['rsi'][i] > 80:
            rsi1curve_result[i] = -1 - abs((rsi14_result['rsi'][i] - 80.0) / 80.0)
            
rsi1curve_result = pd.Series(rsi1curve_result).fillna(0).tolist()

for i in rsi_long_with2curve:
    rsi2curve_result[i] = 1
for i in rsi_short_with2curve:
    rsi2curve_result[i] = -1
    
rsi2curve_result = DeterminedStatus(list(rsi5_result['rsi']), list(rsi14_result['rsi']), rsi2curve_result)
rsi2curve_result = pd.Series(rsi2curve_result).fillna(0).tolist()

In [59]:
# sma
sma_result_5 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 5))
sma_result_10 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 10))
sma_result_5 = sma_result_5.rename(columns={'收盤價(元)':'sma5'})
sma_result_10 = sma_result_10.rename(columns={'收盤價(元)':'sma10'})

sma_result_20 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 20))
sma_result_60 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 60))
sma_result_20 = sma_result_20.rename(columns={'收盤價(元)':'sma20'})
sma_result_60 = sma_result_60.rename(columns={'收盤價(元)':'sma60'})

sma_result_120 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 120))
sma_result_240 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 240))
sma_result_120 = sma_result_120.rename(columns={'收盤價(元)':'sma120'})
sma_result_240 = sma_result_240.rename(columns={'收盤價(元)':'sma240'})

sma_result_5.fillna(value=0, inplace=True)
sma_result_10.fillna(value=0, inplace=True)
sma_result_20.fillna(value=0, inplace=True)
sma_result_60.fillna(value=0, inplace=True)
sma_result_120.fillna(value=0, inplace=True)
sma_result_240.fillna(value=0, inplace=True)

# sma's long points & short points
smas_long = FindTheBuyPoint(sma_result_5['sma5'], sma_result_10['sma10'], "sma")
smas_short = FindTheSellPoint(sma_result_5['sma5'], sma_result_10['sma10'], "sma")

smam_long = FindTheBuyPoint(sma_result_20['sma20'], sma_result_60['sma60'], "sma")
smam_short = FindTheSellPoint(sma_result_20['sma20'], sma_result_60['sma60'], "sma")

smal_long = FindTheBuyPoint(sma_result_120['sma120'], sma_result_240['sma240'], "sma")
smal_short = FindTheSellPoint(sma_result_120['sma120'], sma_result_240['sma240'], "sma")

# create a long/short points list
smas_result = [0 for i in range(len(sma_result_5))]
smam_result = [0 for i in range(len(sma_result_5))]
smal_result = [0 for i in range(len(sma_result_5))]

for i in smas_long:
    smas_result[i] = 1
for i in smas_short:
    smas_result[i] = -1
smas_result = DeterminedStatus(list(sma_result_5['sma5']), list(sma_result_10['sma10']), smas_result)
smas_result = pd.Series(smas_result).fillna(0).tolist()

for i in smam_long:
    smam_result[i] = 1
for i in smam_short:
    smam_result[i] = -1
smam_result = DeterminedStatus(list(sma_result_20['sma20']), list(sma_result_60['sma60']), smam_result)
smam_result = pd.Series(smam_result).fillna(0).tolist()

for i in smal_long:
    smal_result[i] = 1
for i in smal_short:
    smal_result[i] = -1
smal_result = DeterminedStatus(list(sma_result_120['sma120']), list(sma_result_240['sma240']), smal_result)
smal_result = pd.Series(smal_result).fillna(0).tolist()

In [60]:
# macd
MACD_val = pd.DataFrame(MACD(etfPriceData, [12,26,9]))
# kd's long points & short points
MACD_long = FindTheBuyPoint(MACD_val['dif'], MACD_val['dem'], "macd")
MACD_short = FindTheSellPoint(MACD_val['dif'], MACD_val['dem'], "macd")

# create a long/short points list
MACD_result = [0 for i in range(len(MACD_val))]

for i in MACD_long:
    MACD_result[i] = 1
for i in MACD_short:
    MACD_result[i] = -1

MACD_result = DeterminedStatus(list(MACD_val['dif']), list(MACD_val['dem']), MACD_result)
MACD_result = pd.Series(MACD_result).fillna(0).tolist()

In [11]:
sql = "SELECT DATE, CLOSE, HIGH, LOW FROM btc_usd ORDER BY DATE"
pricesData = GetDataFromDB("140.118.126.136", "123", "1234567890", "bitfinex", sql)

In [45]:
pricesData = list(pricesData)
Dates = []
closePrices = []
highPrices = []
lowPrices = []
for i in range(len(pricesData)):
    Dates.append(pricesData[i][0])
    closePrices.append(pricesData[i][1])
    highPrices.append(pricesData[i][2])
    lowPrices.append(pricesData[i][3])
    
prices_df = pd.DataFrame({'Date':Dates, 'Close':closePrices, 'High':highPrices, 'Low':lowPrices})
result = getFeaturesCrossIndicators(prices_df)